# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
# import libraries
import joblib
import pandas as pd
import re

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from time import time

In [2]:
# Create connection to database
engine = create_engine('sqlite:///message_features.db')
con = engine.connect()

# Find the names of tables inside of this database
engine.table_names()

['messages']

In [3]:
# Read in data to pandas dataframe
df = pd.read_sql_table("messages", con)

df.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Create X and Y dataframes
X = df["message"]
Y = df.drop(labels=["message", "original", "genre"], axis=1)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    # Normalise by setting to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Create tokens 
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # Lemmatise words
    clean_tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
basic_logit = LogisticRegression(random_state=42)

basic_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(basic_logit, n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

In [8]:
# Train pipeline
b_pipeline_start_time = time()
basic_pipeline.fit(X_train, Y_train)
b_pipeline_end_time = time()

In [9]:
# Print time to train
print("Time to train:", (b_pipeline_end_time - b_pipeline_start_time)/60, "mins")

Time to train: 2.41788725455602 mins


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
Y_pred = basic_pipeline.predict(X_test)

In [11]:
for i, col in enumerate(Y.columns):
    # if i in [0, 9]: continue  # Skip bad column, TODO: Fix 0th column to not be 3 classes for no reason
    
    print("Column {}: {}".format(i, col))
    
    y_true = list(Y_test.values[:, i])
    y_pred = list(Y_pred[:, i])
    target_names = ['is_{}'.format(col), 'is_not_{}'.format(col)]
    print(classification_report(y_true, y_pred, target_names=target_names))

Column 0: related
                precision    recall  f1-score   support

    is_related       0.74      0.40      0.52      1568
is_not_related       0.84      0.95      0.89      4986

      accuracy                           0.82      6554
     macro avg       0.79      0.68      0.70      6554
  weighted avg       0.81      0.82      0.80      6554

Column 1: request
                precision    recall  f1-score   support

    is_request       0.91      0.98      0.94      5449
is_not_request       0.83      0.53      0.65      1105

      accuracy                           0.90      6554
     macro avg       0.87      0.75      0.79      6554
  weighted avg       0.90      0.90      0.89      6554

Column 2: offer
              precision    recall  f1-score   support

    is_offer       1.00      1.00      1.00      6524
is_not_offer       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\SimonThornewill\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                 precision    recall  f1-score   support

    is_military       0.97      1.00      0.98      6353
is_not_military       0.56      0.09      0.15       201

       accuracy                           0.97      6554
      macro avg       0.77      0.54      0.57      6554
   weighted avg       0.96      0.97      0.96      6554

Column 9: water
              precision    recall  f1-score   support

    is_water       0.96      0.99      0.98      6138
is_not_water       0.78      0.47      0.58       416

    accuracy                           0.96      6554
   macro avg       0.87      0.73      0.78      6554
weighted avg       0.95      0.96      0.95      6554

Column 10: food
              precision    recall  f1-score   support

     is_food       0.95      0.99      0.97      5819
 is_not_food       0.86      0.59      0.70       735

    accuracy                           0.94      6554
   macro avg       0.90      0.79      0.83      6554
weighted avg       0.94 

                   precision    recall  f1-score   support

    is_earthquake       0.97      0.99      0.98      5963
is_not_earthquake       0.89      0.67      0.76       591

         accuracy                           0.96      6554
        macro avg       0.93      0.83      0.87      6554
     weighted avg       0.96      0.96      0.96      6554

Column 32: cold
              precision    recall  f1-score   support

     is_cold       0.98      1.00      0.99      6409
 is_not_cold       0.83      0.10      0.18       145

    accuracy                           0.98      6554
   macro avg       0.91      0.55      0.59      6554
weighted avg       0.98      0.98      0.97      6554

Column 33: other_weather
                      precision    recall  f1-score   support

    is_other_weather       0.95      1.00      0.97      6221
is_not_other_weather       0.57      0.04      0.07       333

            accuracy                           0.95      6554
           macro avg     

### 6. Improve your model
Use grid search to find better parameters. 

<div "I got this implementation from https://github.com/wenshihao1993/Udacity-DSND/blob/master/Data%20Engineering/Disaster%20Response%20Pipeline%20Project/models/train_classifier.py">
</div>

In [12]:
# Define Pipeline
cv_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LogisticRegression(random_state = 42)), n_jobs=-1))
])


In [13]:
# Get parameter names
for i in cv_pipeline.get_params().keys():
    if 'clf' in i: print("\t",i)

	 clf
	 clf__estimator__estimator__C
	 clf__estimator__estimator__class_weight
	 clf__estimator__estimator__dual
	 clf__estimator__estimator__fit_intercept
	 clf__estimator__estimator__intercept_scaling
	 clf__estimator__estimator__l1_ratio
	 clf__estimator__estimator__max_iter
	 clf__estimator__estimator__multi_class
	 clf__estimator__estimator__n_jobs
	 clf__estimator__estimator__penalty
	 clf__estimator__estimator__random_state
	 clf__estimator__estimator__solver
	 clf__estimator__estimator__tol
	 clf__estimator__estimator__verbose
	 clf__estimator__estimator__warm_start
	 clf__estimator__estimator
	 clf__estimator__n_jobs
	 clf__estimator
	 clf__n_jobs


In [14]:
# Create GridSearchCV parameters
parameters = {'clf__estimator__estimator__C': [1, 2, 5],
             'clf__estimator__estimator__penalty': ['l1', 'l2']}

# Instantiate model
gridsearch_pipeline = GridSearchCV(cv_pipeline, param_grid=parameters, scoring='precision_samples', cv = 5)

In [15]:
# Train model
gs_pipeline_start_time = time()
gridsearch_pipeline.fit(X_train, Y_train)
gs_pipeline_end_time = time()

C:\Users\SimonThornewill\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\SimonThornewill\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\SimonThornewill\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\SimonThornewill\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Sim

In [16]:
# Print time to train
print("Time to train:", (gs_pipeline_end_time - gs_pipeline_start_time)/60, "mins")

Time to train: 80.17691243092219 mins


In [17]:
gridsearch_pipeline.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LogisticRegression(C=1,
                                                                                                  class_weight=None,
                     

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
gridsearch_Y_Pred = gridsearch_pipeline.predict(X_test)

In [21]:
for i, col in enumerate(Y.columns):
    # if i in [0, 9]: continue  # Skip bad column, TODO: Fix 0th column to not be 3 classes for no reason
    
    print("Column {}: {}".format(i, col))
    
    y_true = list(Y_test.values[:, i])
    y_pred = list(gridsearch_Y_Pred[:, i])
    target_names = ['is_{}'.format(col), 'is_not_{}'.format(col)]
    print(classification_report(y_true, y_pred, target_names=target_names))

Column 0: related
                precision    recall  f1-score   support

    is_related       0.74      0.40      0.52      1568
is_not_related       0.84      0.95      0.89      4986

      accuracy                           0.82      6554
     macro avg       0.79      0.68      0.70      6554
  weighted avg       0.81      0.82      0.80      6554

Column 1: request
                precision    recall  f1-score   support

    is_request       0.91      0.98      0.94      5449
is_not_request       0.83      0.53      0.65      1105

      accuracy                           0.90      6554
     macro avg       0.87      0.75      0.79      6554
  weighted avg       0.90      0.90      0.89      6554

Column 2: offer
              precision    recall  f1-score   support

    is_offer       1.00      1.00      1.00      6524
is_not_offer       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\SimonThornewill\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

    is_tools       0.99      1.00      1.00      6514
is_not_tools       0.00      0.00      0.00        40

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

Column 23: hospitals
                  precision    recall  f1-score   support

    is_hospitals       0.99      1.00      1.00      6495
is_not_hospitals       0.00      0.00      0.00        59

        accuracy                           0.99      6554
       macro avg       0.50      0.50      0.50      6554
    weighted avg       0.98      0.99      0.99      6554

Column 24: shops
              precision    recall  f1-score   support

    is_shops       1.00      1.00      1.00      6529
is_not_shops       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg

Gridsearch literally had no effect on the outcome when I compare it with the non-gridsearch pipeline

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Notes:
> Since that GridSearch took 80 minutes on my machine at home, I won't be doing any more of that.

> However, I did find that using a random forest worked quite well so I'll quickly train another one of those on the default values and compare with my Logistic Regression model.

In [23]:
forest = RandomForestClassifier(random_state=42)

forest_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(forest, n_jobs=-1))
])

# Train pipeline
b_pipeline_start_time = time()
forest_pipeline.fit(X_train, Y_train)
b_pipeline_end_time = time()

In [28]:
forest_Y_Pred = forest_pipeline.predict(X_test)

In [29]:
for i, col in enumerate(Y.columns):
 
    print("Column {}: {}".format(i, col))
    
    y_true = list(Y_test.values[:, i])
    y_pred = list(forest_Y_Pred[:, i])
    target_names = ['is_{}'.format(col), 'is_not_{}'.format(col)]
    print(classification_report(y_true, y_pred, target_names=target_names))

Column 0: related
                precision    recall  f1-score   support

    is_related       0.62      0.45      0.52      1568
is_not_related       0.84      0.91      0.87      4986

      accuracy                           0.80      6554
     macro avg       0.73      0.68      0.70      6554
  weighted avg       0.79      0.80      0.79      6554

Column 1: request
                precision    recall  f1-score   support

    is_request       0.89      0.98      0.93      5449
is_not_request       0.80      0.43      0.56      1105

      accuracy                           0.89      6554
     macro avg       0.85      0.70      0.75      6554
  weighted avg       0.88      0.89      0.87      6554

Column 2: offer
              precision    recall  f1-score   support

    is_offer       1.00      1.00      1.00      6524
is_not_offer       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

Fancy that, this one seems to do a little worse than the Logistic Regression!

### 9. Export your model as a pickle file

In [32]:
joblib.dump(basic_pipeline, "model.pkl")

['model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.